In [1]:
from __future__ import absolute_import, division

import os
import urllib, cStringIO

import pymongo as pm

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### setting up paths


In [2]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
features_dir= os.path.join(results_dir,'features')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)  

if not os.path.exists(features_dir):
    os.makedirs(features_dir)
    
## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))        
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

<module 'analysis_helpers' from '/Users/judithfan/semantic_parts/analysis/analysis_helpers.pyc'>

#### helpers

In [14]:
def cleanup_df(X):
    if 'Unnamed: 0' in X.columns:
        X = X.drop(columns=['Unnamed: 0'])
    return X

#### load in data

In [15]:
## loading in files we need
feature_vec = np.load(os.path.join(features_dir,'semantic_parts_sketch_features.npy'))
feature_df = cleanup_df(pd.read_csv(os.path.join(csv_dir,'semantic_parts_sketch_features_key.csv')))
meta_df = cleanup_df(pd.read_csv(os.path.join(csv_dir,'semantic_parts_sketch_features_meta.csv')))

In [30]:
## concatenate meta and features to enable easy subsetting of dataframe
F = pd.concat((meta_df,feature_df),axis=1)

In [ ]:
## aggregate by target and condition and take the mean across rows within each group
F2 = F.groupby(['target','condition']).mean().reset_index()

In [60]:
## construct dictionary of object to part labels

## version where each part label within category is treated distinctly
part_dict_expanded = {'bird': ['beak', 'body', 'eye', 'feet', 'head', 'leg', 'tail', 'wing'], \
                     'car': ['body.1', 'bumper', 'door', 'headlight', 'hood', 'trunk', 'wheel', 'window', 'windshield'],\
                     'chair': ['armrest', 'backrest', 'leg.1', 'seat'], \
                     'dog': ['body.2', 'ear', 'eye.1', 'head.1', 'leg.2', 'mouth', 'neck', 'paw', 'tail.1']}

## version where each repeating part labels within category are not distinguished
part_dict_compressed = {'bird': ['beak', 'body', 'eye', 'feet', 'head', 'leg', 'tail', 'wing'], \
                     'car': ['body', 'bumper', 'door', 'headlight', 'hood', 'trunk', 'wheel', 'window', 'windshield'],\
                     'chair': ['armrest', 'backrest', 'leg', 'seat'], \
                     'dog': ['body', 'ear', 'eye', 'head', 'leg', 'mouth', 'neck', 'paw', 'tail']}

## abbreviated handles 
pde = part_dict_expanded
pdc = part_dict_compressed

In [91]:
## add arclength feature to set of columns associated with this category
def add_arclength_cols(category):
    

pde['bird'] = pde['bird'] + [i + '_total_arclength' for i in pde['bird']]

In [93]:
for i in pde['dog']:
    if len(i.split('.'))>1:
        print i.split('.')[0] + '_total_arclength' + i.split('.')[1] 
    else:
        

['body.2', 'ear', 'eye.1', 'head.1', 'leg.2', 'mouth', 'neck', 'paw', 'tail.1']

In [76]:
## get inds for subsetting dataframe
close_inds = F2['condition'] == 'closer'
far_inds = F2['condition'] == 'further'

obj_inds = F2['target']=='tomtit'

In [81]:
## extract particular row corresponding to this object in each condition
f = F2[(far_inds) & obj_inds][pde['bird']].reset_index(drop=True)
c = F2[(close_inds) & obj_inds][pde['bird']].reset_index(drop=True)

In [89]:
[i + '_total_arclength' for i in pde['bird']]

['beak_total_arclength',
 'body_total_arclength',
 'eye_total_arclength',
 'feet_total_arclength',
 'head_total_arclength',
 'leg_total_arclength',
 'tail_total_arclength',
 'wing_total_arclength']

In [83]:
diff = c.sub(f,fill_value=0)

,beak,body,eye,feet,head,leg,tail,wing
0,0.52381,1.349206,0.079365,0.507937,1.460317,-0.84127,0.222222,1.666667


In [84]:
f

,beak,body,eye,feet,head,leg,tail,wing
0,0.47619,1.428571,0.142857,3.047619,0.428571,1.285714,0.333333,0.333333


In [85]:
c

,beak,body,eye,feet,head,leg,tail,wing
0,1.0,2.777778,0.222222,3.555556,1.888889,0.444444,0.555556,2.0


In [88]:
F2.columns

Index([u'target', u'condition', u'outcome', u'None', u'beak', u'body', u'eye',
       u'feet', u'head', u'leg', u'tail', u'unknown', u'wing', u'None.1',
       u'body.1', u'bumper', u'door', u'headlight', u'hood', u'trunk',
       u'unknown.1', u'wheel', u'window', u'windshield', u'None.2', u'armrest',
       u'backrest', u'leg.1', u'seat', u'unknown.2', u'None.3', u'body.2',
       u'ear', u'eye.1', u'head.1', u'leg.2', u'mouth', u'neck', u'paw',
       u'tail.1', u'unknown.3', u'None_total_arclength',
       u'beak_total_arclength', u'body_total_arclength',
       u'eye_total_arclength', u'feet_total_arclength',
       u'head_total_arclength', u'leg_total_arclength',
       u'tail_total_arclength', u'unknown_total_arclength',
       u'wing_total_arclength', u'None_total_arclength.1',
       u'body_total_arclength.1', u'bumper_total_arclength',
       u'door_total_arclength', u'headlight_total_arclength',
       u'hood_total_arclength', u'trunk_total_arclength',
       u'unknown_total